In [2]:
import torch
import torchvision
import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from tqdm import tqdm
import matplotlib.pyplot as plt
from torchvision.datasets import CocoDetection
from torchvision.transforms import ToTensor
import numpy as np
from torch.utils.data import DataLoader,Dataset
import torch.nn as nn
from torch.nn.functional import relu
import copy
import time
from collections import defaultdict
from torch.optim import lr_scheduler
import torch.nn.functional as F
from sklearn.metrics import precision_recall_fscore_support
import pickle

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
! git clone https://github.com/pdollar/coco/
%cd coco/PythonAPI
! make
! python setup.py install
%cd ../..
! rm -r coco
!wget http://images.cocodataset.org/zips/train2017.zip
!wget http://images.cocodataset.org/zips/val2017.zip
!wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip
!unzip \*.zip
!rm *.zip
path2data="./train2017"
path2json="./annotations/instances_train2017.json"

path2data_val="./val2017"
_val="./annotations/instances_val2017.json"

Cloning into 'coco'...
remote: Enumerating objects: 975, done.
remote: Total 975 (delta 0), reused 0 (delta 0), pack-reused 975
Receiving objects: 100% (975/975), 11.72 MiB | 10.48 MiB/s, done.
Resolving deltas: 100% (576/576), done.
/content/coco/PythonAPI
python setup.py build_ext --inplace
running build_ext
Compiling pycocotools/_mask.pyx because it changed.
[1/1] Cythonizing pycocotools/_mask.pyx
/usr/local/lib/python3.10/dist-packages/Cython/Compiler/Main.py:381: FutureWarning: Cython directive 'language_level' not set, using '3str' for now (Py3). This has changed from earlier releases! File: /content/coco/PythonAPI/pycocotools/_mask.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
building 'pycocotools._mask' extension
creating build
creating build/common
creating build/temp.linux-x86_64-cpython-310
creating build/temp.linux-x86_64-cpython-310/pycocotools
x86_64-linux-gnu-gcc -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wform

In [4]:
# Define the COCO training dataset with data transformations
coco_train = torch.utils.data.Subset(datasets.CocoDetection(root=path2data, annFile=path2json, transform=transforms.Compose([
    transforms.Resize((224, 224)),  # Resize the images
    transforms.ToTensor(),  # Convert images to tensors
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalize the image data based on ImageNet statistics
])), [i for i in range(10240)])

# Define another subset of the COCO training dataset with a different size and the same data transformations
coco_train_2 = torch.utils.data.Subset(datasets.CocoDetection(root=path2data, annFile=path2json, transform=transforms.Compose([
    transforms.Resize((224, 224)),  # Resize the images
    transforms.ToTensor(),  # Convert images to tensors
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalize the image data based on ImageNet statistics
])), [i for i in range(4096)])
# Define a reduced subset of the COCO validation dataset with specific transformations and size
coco_val = torch.utils.data.Subset(datasets.CocoDetection(root=path2data_val, annFile=_val, transform=transforms.Compose([
    transforms.Resize((224, 224)),  # Resize the images
    transforms.ToTensor(),  # Convert images to tensors
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalize the image data based on ImageNet statistics
])), [i for i in range(2000)])


loading annotations into memory...


Exception ignored in: <function _xla_gc_callback at 0x7bf0088d2050>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/jax/_src/lib/__init__.py", line 97, in _xla_gc_callback
    def _xla_gc_callback(*args):
KeyboardInterrupt: 
Exception ignored in: <function _xla_gc_callback at 0x7bf0088d2050>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/jax/_src/lib/__init__.py", line 97, in _xla_gc_callback
    def _xla_gc_callback(*args):
KeyboardInterrupt: 


Done (t=22.20s)
creating index...
index created!
loading annotations into memory...
Done (t=21.75s)
creating index...
index created!
loading annotations into memory...
Done (t=0.62s)
creating index...
index created!


In [18]:
# Custom PyTorch dataset class for COCO dataset
class CocoDataset(Dataset):
    def __init__(self, num_objects, dataset):
        """
        Constructor for the CocoDataset class.

        Parameters:
        - num_objects (int): Number of object categories for one-hot encoding.
        - dataset: Original COCO dataset.
        """
        self.coco_dataset = dataset
        self.num_objects = num_objects

    def convert_to_one_hot(self, numbers, size):
        """
        Convert a list of numbers to one-hot encoded tensor.

        Parameters:
        - numbers (list): List of numbers to be one-hot encoded.
        - size (int): Size of the one-hot encoded tensor.

        Returns:
        - torch.Tensor: One-hot encoded tensor.
        """
        return torch.from_numpy(np.eye(size)[numbers])

    def __getitem__(self, index):
        """
        Get a single item from the dataset.

        Parameters:
        - index (int): Index of the item.

        Returns:
        - torch.Tensor: Placeholder 'data' (actual data loading implementation is missing).
        - torch.Tensor: One-hot encoded labels for object categories.
        """
        # Note: The 'data' placeholder should be replaced with actual data loading code
        data, target = self.coco_dataset[index];
        return data, self.convert_to_one_hot([target[i]['category_id']-1 for i in range(len(target))], self.num_objects)

    def __len__(self):
        """
        Get the length of the dataset.

        Returns:
        - int: Length of the dataset.
        """
        return len(self.coco_dataset)


In [19]:
# Create data loaders for training and testing datasets
train_data_loader1 = DataLoader(CocoDataset(90, coco_train), batch_size=64, shuffle=True)  # Training data loader 1
train_data_loader2 = DataLoader(CocoDataset(90, coco_train_2), batch_size=64, shuffle=True)  # Training data loader 2
test_data_loader = DataLoader(CocoDataset(90, coco_val), batch_size=64, shuffle=True)  # Testing data loader


In [20]:
# Custom Deformable Convolutional Layer using deformable convolution operation
class DeformableConv2d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, padding=1, dilation=1, deformable_groups=1):
        super(DeformableConv2d, self).__init__()

        # Set convolutional layer parameters
        self.kernel_size, self.stride, self.padding, self.dilation = (kernel_size, kernel_size), (stride, stride), padding, dilation

        # Define the deformable convolutional layer components
        self.modulator, self.offset_conv, self.final_conv = nn.Sequential(
            nn.Conv2d(in_channels, deformable_groups, kernel_size=self.kernel_size, stride=stride, padding=self.padding, dilation=self.dilation, bias=False),
            nn.Conv2d(in_channels, deformable_groups * 2, kernel_size=self.kernel_size, stride=stride, padding=self.padding, dilation=self.dilation, bias=False),
            nn.Conv2d(in_channels, out_channels, kernel_size=self.kernel_size, stride=stride, padding=self.padding, dilation=self.dilation, bias=False)
        )

    def deformed_feature(self, x):
        """
        Generate deformed feature maps using deformable convolution operation.

        Parameters:
        - x (torch.Tensor): Input feature map.

        Returns:
        - torch.Tensor: Deformed feature map.
        """
        N, C, H, W = x.shape

        # Create a grid for offset calculation
        grid_y, grid_x = torch.meshgrid(torch.arange(H).to(device=x.device), torch.arange(W).to(device=x.device))

        # Calculate offset using offset convolution
        grid_y, grid_x = (grid_y.view(1, H, W) + self.offset_conv(x)[:, 0, :, :]), (grid_x.view(1, H, W) + self.offset_conv(x)[:, 1, :, :])

        # Interpolate features using bilinear interpolation
        grid = torch.stack((2 * grid_x / (W - 1) - 1, 2 * grid_y / (H - 1) - 1), dim=-1)
        features = nn.functional.grid_sample(x, grid, mode='bilinear', align_corners=False)

        # Apply modulation to the deformed features
        return features * 2 * torch.sigmoid(self.modulator(x))

    def forward(self, x):
        """
        Forward pass through the deformable convolutional layer.

        Parameters:
        - x (torch.Tensor): Input feature map.

        Returns:
        - torch.Tensor: Output feature map.
        """
        return self.final_conv(self.deformed_feature(x))

# Instantiate a Deformable Convolutional Layer
deformable_conv = DeformableConv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1, deformable_groups=1)

# Generate random input data
test = torch.randn(6, 32, 224, 224)  # (#samples, #in channels, #heights, #width)

# Perform a forward pass through the Deformable Convolutional Layer
out = deformable_conv(test)


In [21]:
# Object Detector Neural Network Module
class ObjectDetector(nn.Module):
    def __init__(self, n_class, deformable=False):
        """
        Constructor for the ObjectDetector class.

        Parameters:
        - n_class (int): Number of object classes.
        - deformable (bool): Flag indicating whether to use deformable convolutional layers.
        """
        super(ObjectDetector, self).__init__()  # Use super() to call the nn.Module constructor
        conv = nn.Conv2d if not deformable else DeformableConv2d


        # Define the blocks of the neural network architecture
        self.e11, self.e12, self.pool1 = nn.Sequential(
            conv(3, 64, kernel_size=3, padding=1),
            conv(64, 64, kernel_size=3, padding=1),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.e21, self.e22, self.pool2 = nn.Sequential(
            conv(64, 128, kernel_size=3, padding=1),
            conv(128, 128, kernel_size=3, padding=1),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.e31, self.e32, self.pool3 = nn.Sequential(
            conv(128, 256, kernel_size=3, padding=1),
            conv(256, 256, kernel_size=3, padding=1),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.e41, self.e42, self.pool4 = nn.Sequential(
            conv(256, 512, kernel_size=3, padding=1),
            conv(512, 512, kernel_size=3, padding=1),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.e51, self.e52 = conv(512, 1024, kernel_size=3, padding=1), conv(1024, 1024, kernel_size=3, padding=1)

        # Output layer components
        self.outconv, self.fc_layers, self.BN, self.sigmoid = nn.Conv2d(1024, n_class, kernel_size=1), nn.Sequential(
            nn.Linear(17640, 1024), nn.ReLU(),
            nn.Linear(1024, 256), nn.ReLU(),
            nn.Linear(256, n_class)
        ), nn.BatchNorm1d(n_class), nn.Sigmoid()

    def forward(self, x):
        """
        Forward pass through the Object Detector network.

        Parameters:
        - x (torch.Tensor): Input tensor.

        Returns:
        - torch.Tensor: Output tensor.
        """
        # Forward pass through the network architecture
        out = self.outconv(relu(self.e52(relu(self.e51(self.pool4(relu(self.e42(
            relu(self.e41(self.pool3(relu(self.e32(relu(self.e31(
                self.pool2(relu(self.e22(relu(self.e21(self.pool1(
                    relu(self.e12(relu(self.e11(x)))))))))))))))))))))))))

        # Apply batch normalization, sigmoid activation, and fully connected layers to the flattened output
        return self.sigmoid(self.BN(self.fc_layers(out.view(out.shape[0], -1))))


In [23]:
def train_model(model, optimizer, scheduler, num_epochs=20):
    """
    Train the given model using the specified optimizer and scheduler for a certain number of epochs.

    Parameters:
        model (torch.nn.Module): The model to be trained.
        optimizer (torch.optim.Optimizer): The optimizer used for updating model parameters.
        scheduler (torch.optim.lr_scheduler._LRScheduler): The learning rate scheduler.
        num_epochs (int): Number of epochs for training.

    Returns:
        torch.nn.Module: The trained model.
        dict: Output metrics including precision, recall, F1 score, and loss for both training and validation sets.
    """
    output_metrics = {'f1': {'train': [], 'val': []},
                      'precision': {'train': [], 'val': []},
                      'recall': {'train': [], 'val': []},
                      'loss': {'train': [], 'val': []}
                      }

    all_metrics = ['precision', 'recall', 'f1', 'loss']

    dataloaders = {
        'train': train_data_loader1,
        'val': test_data_loader}
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = 1e10

    epoch_loss = 0
    losses = []
    criterion = torch.nn.BCELoss()

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 20)

        since = time.time()

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                for param_group in optimizer.param_groups:
                    print("LR", param_group['lr'])

                model.train()  # Set model to training mode
            else:
                model.eval()  # Set model to evaluate mode

            metrics = defaultdict(float)
            epoch_samples = 0

            all_predicted = []
            all_targets = []
            for index, (inputs, labels) in enumerate(tqdm(dataloaders[phase])):
                inputs = inputs.to(device).float()
                labels = labels.to(device).float()

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    metrics['loss'] += loss.data.cpu().numpy() * labels.size(0)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                predicted = (outputs > 0.5).float()
                all_predicted.extend(predicted.cpu().numpy().flatten())
                all_targets.extend(labels.cpu().numpy().flatten())

                epoch_samples += inputs.size(0)

            temp = [0, 0, 0, 0]
            temp[0], temp[1], temp[2], _ = precision_recall_fscore_support(all_targets, all_predicted,
                                                                           average='binary')
            temp[3] = metrics['loss'] / epoch_samples
            for index, metric in enumerate(all_metrics):
                output_metrics[metric][phase].append(temp[index])

            outputs = []
            for k in metrics.keys():
                outputs.append("{}: {:4f}".format(k, metrics[k] / epoch_samples))
            print("{}: {}".format(phase, ", ".join(outputs)))
            epoch_loss = metrics['loss'] / epoch_samples

            # deep copy the model
            if phase == 'val' and epoch_loss < best_loss:
                print("saving best model")
                best_loss = epoch_loss
                best_model_wts = copy.deepcopy(model.state_dict())

        time_elapsed = time.time() - since
        print('{:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))

        # save the checkpoint
        checkpoint = {
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
        }
        torch.save(checkpoint, f'/content/drive/My Drive/DEEP/HW3/Q2/checkpoints/model_checkpoint{epoch}.pth')

        # Save the dictionary to a file
        with open('/content/drive/My Drive/DEEP/dictionary_conv_deform.pickle', 'wb') as file:
            pickle.dump(output_metrics, file)

    print('Best val loss: {:4f}'.format(best_loss))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, output_metrics


# Instantiate the ObjectDetector model
model = ObjectDetector(90).to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))

# Define the learning rate and alpha (smoothing constant)
learning_rate = 1e-3
alpha = 0.9  # You can adjust this value based on your needs

# Define the RMSprop optimizer with alpha
optimizer_ft = torch.optim.RMSprop(model.parameters(), lr=learning_rate, alpha=alpha)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=30, gamma=0.1)

# Train the model using the defined parameters
ObjectDetector_model, output_metrics_cov = train_model(model, optimizer_ft, exp_lr_scheduler, num_epochs=50)


Epoch 0/49
--------------------
LR 0.001


  0%|          | 0/160 [00:01<?, ?it/s]


RuntimeError: ignored